In [90]:
import regex as re

In [91]:
with open("12/input.txt") as f:
    lines = f.read().splitlines()
lines = [line.split() for line in lines]
line_constraints: list[tuple[str, list[int]]] = [(line, [int(x) for x in blocks.split(",")]) for line, blocks in lines]
line_constraints[:5]

[('..???.??.?', [1, 1, 1]),
 ('?#?##???.????', [2, 5, 1, 1]),
 ('?#??????##?', [1, 1, 2]),
 ('?#.#?#??#???', [1, 7, 1]),
 ('?#???#?#??.#.###.?', [3, 1, 3, 1, 3, 1])]

In [92]:
def expand_line(line: str, contiguous_broken: list[int], times=5) -> tuple[str, list[int]]:
    return "?".join([line] * times), contiguous_broken * times

In [93]:
expanded_constraints = [
    expand_line(*constraint, times=5)
    for constraint in line_constraints
]
expanded_constraints[:5]

[('..???.??.??..???.??.??..???.??.??..???.??.??..???.??.?',
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('?#?##???.??????#?##???.??????#?##???.??????#?##???.??????#?##???.????',
  [2, 5, 1, 1, 2, 5, 1, 1, 2, 5, 1, 1, 2, 5, 1, 1, 2, 5, 1, 1]),
 ('?#??????##???#??????##???#??????##???#??????##???#??????##?',
  [1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2]),
 ('?#.#?#??#?????#.#?#??#?????#.#?#??#?????#.#?#??#?????#.#?#??#???',
  [1, 7, 1, 1, 7, 1, 1, 7, 1, 1, 7, 1, 1, 7, 1]),
 ('?#???#?#??.#.###.???#???#?#??.#.###.???#???#?#??.#.###.???#???#?#??.#.###.???#???#?#??.#.###.?',
  [3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1])]

In [94]:
def get_block_lengths(line: str) -> list[int]:
    blocks = [block for block in line.replace("?", ".").split(".") if len(block) > 0]
    block_lengths = [len(block) for block in blocks]
    return block_lengths

def satisfied(line: str, contiguous_broken: list[int]) -> bool:
    return get_block_lengths(line) == contiguous_broken

In [95]:
CACHE = {}
def solutions_to_place_blocks(line: str, blocks: list[int], num_hashes: int = 0, expected_num_hashes: int = 0) -> int:
    
    if num_hashes != expected_num_hashes:
        return 0

    blocks_key = "".join(map(str, blocks))
    if (line, blocks_key) in CACHE:
        return CACHE[(line, blocks_key)]

    match blocks:
        case [block, *rest]:
            pattern = "[.?][?#]{" + str(block) + "}[.?]"

            if len(rest) != 0:
                has_to_fit = sum(rest) + len(rest) - 1
                rest_fit = len(line) - has_to_fit

                before, after = line[:rest_fit], line[rest_fit:]
                before_extended = f".{before}"
    
            else:
                before_extended = f".{line}."
                after = ""


            num = 0
        
            for x in re.finditer(pattern, before_extended, overlapped=True):
                span_start, span_end = x.span()
                
                if len(rest) != 0:
                    before = before_extended[span_end:]
                else:
                    before = before_extended[span_end:-1]
                    
                to_solve = before + after
                sols = solutions_to_place_blocks(to_solve, rest, num_hashes + before_extended[:span_start].count("#") + block, expected_num_hashes + block)
                num += sols
            result = num
        
        case []:
            if "#" in line:
                result = 0
            else:
                result = 1

    CACHE[(line, blocks_key)] = result
    return result

In [96]:
sols = [solutions_to_place_blocks(*c) for c in line_constraints]
sum(sols)

8419

In [97]:
expanded_constraints

[('..???.??.??..???.??.??..???.??.??..???.??.??..???.??.?',
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('?#?##???.??????#?##???.??????#?##???.??????#?##???.??????#?##???.????',
  [2, 5, 1, 1, 2, 5, 1, 1, 2, 5, 1, 1, 2, 5, 1, 1, 2, 5, 1, 1]),
 ('?#??????##???#??????##???#??????##???#??????##???#??????##?',
  [1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2]),
 ('?#.#?#??#?????#.#?#??#?????#.#?#??#?????#.#?#??#?????#.#?#??#???',
  [1, 7, 1, 1, 7, 1, 1, 7, 1, 1, 7, 1, 1, 7, 1]),
 ('?#???#?#??.#.###.???#???#?#??.#.###.???#???#?#??.#.###.???#???#?#??.#.###.???#???#?#??.#.###.?',
  [3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1]),
 ('?#......#.?.?.??#......#.?.?.??#......#.?.?.??#......#.?.?.??#......#.?.?.',
  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 ('.????#??????.????#??????.????#??????.????#??????.????#?????',
  [2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2,

In [98]:
from tqdm import tqdm
sols = [solutions_to_place_blocks(*c) for c in tqdm(expanded_constraints)]
sum(sols)

100%|██████████| 1000/1000 [00:01<00:00, 941.61it/s]


160500973317706